<a href="https://colab.research.google.com/github/olesia-za/python_for_ds_tasks/blob/main/OZ_Done_M16_HW2_Credit_risk_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнє завдання: Прогнозування кредитного ризику

## Мета завдання
Застосувати знання з лекції для побудови моделі логістичної регресії, що прогнозує ймовірність дефолту за кредитом, іншими словами, що людина не поверне кредит. Ви пройдете весь цикл: від дослідницького аналізу до оцінки якості класифікаційної моделі.

## Опис датасету
**Credit Risk Dataset** містить інформацію про 32,000+ позичальників з такими параметрами:
- **person_age**: Вік позичальника
- **person_income**: Річний дохід
- **person_home_ownership**: Тип володіння житлом
- **person_emp_length**: Стаж роботи (в роках)
- **loan_intent**: Мета кредиту
- **loan_grade**: Кредитний рейтинг (A - кращий, F - гірший)
- **loan_amnt**: Сума кредиту
- **loan_int_rate**: Процентна ставка
- **loan_status**: Статус кредиту (0 = сплачено, 1 = дефолт) - **цільова змінна**
- **loan_percent_income**: Відношення кредиту до доходу
- **cb_person_default_on_file**: Історія дефолтів (Y/N)
- **cb_person_cred_hist_length**: Довжина кредитної історії

---

## Завдання 1: Завантаження та перший огляд даних (1 бал)

**Що потрібно зробити:**
1. Завантажте дані з файлу `credit_risk_dataset.csv`
2. Виведіть розмір датасету
3. Покажіть перші 5 рядків
4. Виведіть загальну інформацію про дані (кількість записів, типи колонок)
5. Перевірте розподіл цільової змінної (відсотк даних для кожного класу)

Дайте висновок, це задача збалансованої чи незбалансованої класифікації.


In [43]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


df = pd.read_csv('../data/credit_risk_dataset.csv')
df.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4


In [44]:
df.info(), df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32581 entries, 0 to 32580
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_age                  32581 non-null  int64  
 1   person_income               32581 non-null  int64  
 2   person_home_ownership       32581 non-null  object 
 3   person_emp_length           31686 non-null  float64
 4   loan_intent                 32581 non-null  object 
 5   loan_grade                  32581 non-null  object 
 6   loan_amnt                   32581 non-null  int64  
 7   loan_int_rate               29465 non-null  float64
 8   loan_status                 32581 non-null  int64  
 9   loan_percent_income         32581 non-null  float64
 10  cb_person_default_on_file   32581 non-null  object 
 11  cb_person_cred_hist_length  32581 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 3.0+ MB


(None, (32581, 12))

In [45]:
df['loan_status'].value_counts(normalize=True).round(3)
# це задача незбалансованої класифікації, так як не рівномірний розподіл класів цільової змінної

loan_status
0    0.782
1    0.218
Name: proportion, dtype: float64


---

## Завдання 2: Дослідницький аналіз даних (EDA) (4 бали)

**Що потрібно зробити:**
1. Обчисліть відсоток пропущених значень в колонках. За наявності пропущених значень - заповніть їх медіаною для числових колонок і найбільш частим значеннмя для категоріальних.
2. Проаналізуйте розподіл числових змінних.
3. Знайдіть та обробіть викиди в колонці person_income з допомогою [Interquartile range](https://uk.wikipedia.org/wiki/%D0%9C%D1%96%D0%B6%D0%BA%D0%B2%D0%B0%D1%80%D1%82%D0%B8%D0%BB%D1%8C%D0%BD%D0%B8%D0%B9_%D1%80%D0%BE%D0%B7%D0%BC%D0%B0%D1%85).
4. Проаналізуйте категоріальні змінні відносно цільової та частоту зустрічання різних значень в них.
5. Візуалізуйте взаємозв'язок ознак з цільовою змінною.


In [46]:
missing = df.isnull().sum()
missing_percent = (missing / len(df)) * 100
missing_percent[missing > 0].round(2)

person_emp_length    2.75
loan_int_rate        9.56
dtype: float64

In [47]:
df['person_emp_length'] = df['person_emp_length'].fillna(df['person_emp_length'].median())
df['loan_int_rate'] = df['loan_int_rate'].fillna(df['loan_int_rate'].median())

In [48]:
df.select_dtypes('number').describe().round(3)

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_cred_hist_length
count,32581.000,32581.000,32581.000,32581.000,32581.000,32581.000,32581.000,32581.000
mean,27.735,66074.848,4.768,9589.371,11.010,0.218,0.170,5.804
std,6.348,61983.119,4.087,6322.087,3.082,0.413,0.107,4.055
min,20.000,4000.000,0.000,500.000,5.420,0.000,0.000,2.000
25%,23.000,38500.000,2.000,5000.000,8.490,0.000,0.090,3.000
50%,26.000,55000.000,4.000,8000.000,10.990,0.000,0.150,4.000
75%,30.000,79200.000,7.000,12200.000,13.110,0.000,0.230,8.000
max,144.000,6000000.000,123.000,35000.000,23.220,1.000,0.830,30.000


In [49]:
Q1 = df['person_income'].quantile(0.25)
Q3 = df['person_income'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR    
df = df[(df['person_income'] >= lower_bound) & (df['person_income'] <= upper_bound)]
df.select_dtypes('number').describe().round(3)

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_cred_hist_length
count,31097.000,31097.000,31097.000,31097.000,31097.000,31097.000,31097.000,31097.000
mean,27.599,58705.000,4.688,9289.717,10.995,0.224,0.174,5.726
std,6.197,27519.057,3.961,6038.671,3.078,0.417,0.107,4.007
min,20.000,4000.000,0.000,500.000,5.420,0.000,0.010,2.000
25%,23.000,38000.000,2.000,5000.000,8.490,0.000,0.090,3.000
50%,26.000,54000.000,4.000,8000.000,10.990,0.000,0.150,4.000
75%,30.000,75000.000,7.000,12000.000,13.110,0.000,0.230,8.000
max,123.000,140004.000,123.000,35000.000,23.220,1.000,0.830,30.000


In [50]:
df.select_dtypes(include=['object']).nunique()

person_home_ownership        4
loan_intent                  6
loan_grade                   7
cb_person_default_on_file    2
dtype: int64

In [51]:
df.select_dtypes("number").describe().round(3)

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_cred_hist_length
count,31097.000,31097.000,31097.000,31097.000,31097.000,31097.000,31097.000,31097.000
mean,27.599,58705.000,4.688,9289.717,10.995,0.224,0.174,5.726
std,6.197,27519.057,3.961,6038.671,3.078,0.417,0.107,4.007
min,20.000,4000.000,0.000,500.000,5.420,0.000,0.010,2.000
25%,23.000,38000.000,2.000,5000.000,8.490,0.000,0.090,3.000
50%,26.000,54000.000,4.000,8000.000,10.990,0.000,0.150,4.000
75%,30.000,75000.000,7.000,12000.000,13.110,0.000,0.230,8.000
max,123.000,140004.000,123.000,35000.000,23.220,1.000,0.830,30.000


In [52]:
# Кредитний статус за власністю житла
ownership_loan_status = df.groupby(['person_home_ownership', 'loan_status']).size().unstack()
ownership_loan_status_pct = ownership_loan_status.div(ownership_loan_status.sum(axis=1), axis=0) * 100

fig = px.bar(
    ownership_loan_status_pct.T,
    title='Кредитний статус за власністю житла',
    labels={'value': 'Відсоток (%)', 'index': 'Кредитний статус'},
    color_discrete_map={'No':'#2ecc71', 'Yes':'#e74c3c'},
    barmode='group'
)
fig.show()

In [53]:
# Кредитний статус за власністю житла
ownership_loan_status = df.groupby(['loan_intent', 'loan_status']).size().unstack()
ownership_loan_status_pct = ownership_loan_status.div(ownership_loan_status.sum(axis=1), axis=0) * 100

fig = px.bar(
    ownership_loan_status_pct.T,
    title='Кредитний статус за метою кредиту',
    labels={'value': 'Відсоток (%)', 'index': 'Кредитний статус'},
    color_discrete_map={'No':'#2ecc71', 'Yes':'#e74c3c'},
    barmode='group'
)
fig.show()

In [54]:
# Кредитний статус за кредитним рейтингом
ownership_loan_status = df.groupby(['loan_grade', 'loan_status']).size().unstack()
ownership_loan_status_pct = ownership_loan_status.div(ownership_loan_status.sum(axis=1), axis=0) * 100

fig = px.bar(
    ownership_loan_status_pct.T,
    title='Кредитний статус за кредитним рейтингом',
    labels={'value': 'Відсоток (%)', 'index': 'Кредитний статус'},
    color_discrete_map={'No':'#2ecc71', 'Yes':'#e74c3c'},
    barmode='group'
)
fig.show()

In [56]:
# Кредитний статус за історією дефолтів
ownership_loan_status = df.groupby(['cb_person_default_on_file', 'loan_status']).size().unstack()
ownership_loan_status_pct = ownership_loan_status.div(ownership_loan_status.sum(axis=1), axis=0) * 100

fig = px.bar(
    ownership_loan_status_pct.T,
    title='Кредитний статус за історією дефолтів',
    labels={'value': 'Відсоток (%)', 'index': 'Кредитний статус'},
    color_discrete_map={'No':'#2ecc71', 'Yes':'#e74c3c'},
    barmode='group'
)
fig.show()

In [62]:
# Кредитний статус vs дохід
fig = px.box(
    df,
    x='loan_status',
    y='person_income',
    title='Кредитний статус vs дохід',
    labels={'person_income': 'Дохід', 'loan_status': 'Кредитний статус'},
    color='loan_status',
    color_discrete_map={'No':'#2ecc71', 'Yes':'#e74c3c'}
)
fig.show()

In [61]:
# Кредитний статус vs cума кредиту
fig = px.box(
    df,
    x='loan_status',
    y='loan_amnt',
    title='Кредитний статус vs cума кредиту',
    labels={'loan_amnt': 'Сума кредиту', 'loan_status': 'Кредитний статус'},
    color='loan_status',
    color_discrete_map={'No':'#2ecc71', 'Yes':'#e74c3c'}
)
fig.show()

In [64]:
# Кредитний статус vs відсоткова ставка
fig = px.box(
    df,
    x='loan_status',
    y='loan_int_rate',
    title='Кредитний статус vs відсоткова ставка',
    labels={'loan_int_rate': 'Відсоткова ставка', 'loan_status': 'Кредитний статус'},
    color='loan_status',
    color_discrete_map={'No':'#2ecc71', 'Yes':'#e74c3c'}
)
fig.show()

In [66]:
# Кредитний статус vs відношення кредиту до доходу
fig = px.box(
    df,
    x='loan_status',
    y='loan_percent_income',
    title='Кредитний статус vs відношення кредиту до доходу',
    labels={'loan_percent_income': 'Відношення кредиту до доходу', 'loan_status': 'Кредитний статус'},
    color='loan_status',
    color_discrete_map={'No':'#2ecc71', 'Yes':'#e74c3c'}
)
fig.show()

In [ ]:
## Опис датасету
**Credit Risk Dataset** містить інформацію про 32,000+ позичальників з такими параметрами:
- **person_age**: Вік позичальника
- **person_income**: Річний дохід
- **person_home_ownership**: Тип володіння житлом
- **person_emp_length**: Стаж роботи (в роках)
- **loan_intent**: Мета кредиту
- **loan_grade**: Кредитний рейтинг (A - кращий, F - гірший)
- **loan_amnt**: Сума кредиту
- **loan_int_rate**: Процентна ставка
- **loan_status**: Статус кредиту (0 = сплачено, 1 = дефолт) - **цільова змінна**
- **loan_percent_income**: Відношення кредиту до доходу
- **cb_person_default_on_file**: Історія дефолтів (Y/N)
- **cb_person_cred_hist_length**: Довжина кредитної історії

---


---

## Завдання 3: Аналіз кореляцій та Feature Engineering (3 бали)

**Що потрібно зробити:**
1. Побудуйте матрицю кореляцій для числових змінних.
2. Закодуйте категоріальні змінні.
3. Виберіть фінальний набір ознак, можна лишити всі, якщо ви вважаєте, що це - доцільно.




---

## Завдання 4: Підготовка даних та навчання моделі (3 бали)

**Що потрібно зробити:**
1. Розділіть дані на X та y
2. Поділіть на навчальну та тестову вибірки
3. Застосуйте масштабування
4. Навчіть модель логістичної регресії
5. Зробіть прогнози на тренувальній та тестовій вибірках.




---

## Завдання 5: Оцінка якості моделі (4 бали)

**Що потрібно зробити:**
1. Побудуйте confusion matrix.
2. Обчисліть основні метрики (accuracy, precision, recall, f1).
3. Побудуйте ROC-криву та обчисліть AUC
4. Проаналізуйте важливість ознак.
5. Зробіть висновки про якість моделі та які ознаки найбільше впливають на прогноз.



---

## Завдання 6: Оптимізація порогу та бізнес-аналіз (2 бали)

**Що потрібно зробити:**
1. Проаналізуйте метрики precision, recall, F1 при різних порогах класифікації (мінімум - 5 різних порогів). Візуалізуйте як змінюються метрики якості при зміні порогу.
2. Оберіть оптимальний поріг для бізнес-задачі - можна обрати виходячи з попереднього пункту, або додати своїх роздумів і обрати інший.




---

## Завдання 7 (Опціональне): Покращення моделі та висновки (2 бали)

**Що потрібно зробити:**

На цих же даних навчіть DecisionTreeClassifier та RandomForestClassifier. Яка з трьох моделей дає найкращий результат на тестовому наборі?

Поріг класифікації можна для порівняння якостей моделей використовувати стандартний 0.5, або той, що ви виявили, як оптимальний на попередньому кроці.
